# Experiment D2 
- Uma quebra no D para reimplemtação do SVD sem o módulo Surprise

Ideas:

1) Reimplementar o SVD.

2) Implementar um framework de busca de hiperparâmetros.

2.1) N fatores (`n_factors`) da decomposição FM.

2.2) N top colunas (`top_cols`) do dataset.

2.3) Parâmetro $L$ (`recomender(...,L,...)`).


- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [2]:
from typing import NewType, List
import functools
import operator
from collections import Counter, defaultdict
from copy import deepcopy
from time import time

import numpy as np
import pandas as pd

from loguru import logger
from tqdm import tqdm

import pythran
from IPython.display import display, Markdown

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

import jax.numpy as npj
from jax import jit

In [3]:
# Run this cell before close.
%watermark
%watermark -p loguru
%watermark -p pythran
%watermark -p pythran
%watermark -p jax
%watermark -p scipy
%watermark --iversion
%watermark -b -r -g

2020-06-27T20:11:32+00:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 8.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit
loguru 0.5.1
pythran 0.9.5
pythran 0.9.5
jax 0.1.71
scipy 1.4.1
pythran 0.9.5
pandas  1.0.5
numpy   1.19.0

Git hash: 6aae247f20fe42b71b7beb759311d36150482667
Git repo: https://github.com/ysraell/aceleradev_private.git
Git branch: master


From exp. D2:

In [4]:

logger.info("Carregando e processando o dataset...")

path_data = '../data/'
top_cols = pd.read_csv('top_cols.csv')['cols'].to_list()
df_marked = pd.read_csv(path_data+'estaticos_market.csv', usecols=top_cols)
col_user = 'id'
top_cols.remove(col_user)

rest_cols = []
for col in top_cols:
    df_marked[col] = df_marked[col].fillna(0)*1
    
def normalize(x):
    return (x-np.min(x))/(np.max(x) - np.min(x)) if (np.max(x) - np.min(x)) > 0 else (x-np.min(x))

escala = 255
for col in top_cols:
    try:
        df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    except:
        maping = {val:i+1 for i,val in enumerate(df_marked[col].unique())}
        df_marked[col] = df_marked[col].apply(lambda x: maping[x])
        df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    
remove_cols = []
for col in top_cols:
    if df_marked[col].nunique() == 1:
        remove_cols.append(col)

df_marked = df_marked.drop(columns=remove_cols)
for col in remove_cols:
    top_cols.remove(col)

df_ep_list = [pd.read_csv(path_data+'estaticos_portfolio{}.csv'.format(i+1)) for i in range(3)]
tmp = []
for i in range(3):
    df_ep_list[i]['P'] = i+1 
    tmp.append(df_ep_list[i][['id','P']])
df_ep = pd.concat(tmp)
del df_ep_list
del tmp

logger.info("...pronto!")

2020-06-27 20:11:35.602 | INFO     | __main__:<module>:1 - Carregando e processando o dataset...
2020-06-27 20:11:59.894 | INFO     | __main__:<module>:43 - ...pronto!


In [5]:
!ls *.so && rm -f *.so

pythranized_b9af09ce3bb3a83c5d007fe0e852e1cd.cpython-37m-x86_64-linux-gnu.so


In [6]:
%load_ext pythran.magic

In [7]:
%%pythran -fopenmp
#pythran export normalizeitor(float64[][])
#pythran export transformer(float64[][])
#pythran export vector_distance_pythran(int8[][],int8[])
#pythran export pairwise_distance_pythran(int8[][])
#pythran export pairs_distance_pythran(int8[][],int8[][])

def normalizeitor(x):
    return (x -x.min())/(x.max() - x.min() +1e-10)

def transformer(U):
    for i in range(len(U)):
        U[i] = 127*normalizeitor(U[i])
    return U

def vector_distance_pythran(X,vec):
    return abs(X - vec).sum(-1)

def pairwise_distance_pythran(X):
    i = X.shape[0]
    j = X.shape[1]
    return abs(X.reshape((i, 1, j)) - X).sum(-1)

def pairs_distance_pythran(X,Y):
    i = X.shape[0]
    j = X.shape[1]
    return abs(X.reshape((i, 1, j)) - Y).sum(-1)



In [8]:
!ls *.so

pythranized_b9af09ce3bb3a83c5d007fe0e852e1cd.cpython-37m-x86_64-linux-gnu.so


In [9]:
def flat(a):
    return functools.reduce(operator.iconcat, a, []) 

In [10]:
M = df_marked[df_marked.columns[1:]].values.astype(np.int8)

In [11]:
M.shape

(462298, 86)

In [12]:
type(M)

numpy.ndarray

In [13]:
def np_svd(M):
    u, _, _ = np.linalg.svd(M, full_matrices=False)
    return u

def sp_svds(M,k=10):
    A = csc_matrix(M, dtype=float)
    u, _, _ = svds(A, k=k)
    return u

In [ ]:
%timeit sp_svds(M,k=85)
%timeit np_svd(M)

In [14]:
def np_svd_j(M):
    u, _, _ = npj.linalg.svd(M, full_matrices=False)
    return u

In [15]:
np_svd_jit = jit(np_svd_j)

In [ ]:
times_list = []
for _ in range(100):
    ts = [0, 0]
    np.random.shuffle(M)
    t = time()
    u = np_svd(M)
    ts[0] = time()-t
    t = time()
    u = np_svd_jit(M)
    ts[1] = time()-t
    times_list.append(ts)

In [ ]:
print(pd.DataFrame(times_list, columns=['NP','NP/JAX/JIT']).describe())

In [16]:
def vector_distance(X,vec):
    return abs(X - vec).sum(-1)

def pairwise_distance(X):
    return abs(X[:, None, :] - X).sum(-1)

def pairs_distance(X,Y):
    return abs(X[:, None, :] - Y).sum(-1)

def vector_distance_float(X,vec):
    return abs((X - vec).astype(npj.float32)).sum(-1)

def pairwise_distance_float(X):
    return abs((X[:, None, :] - X).astype(npj.float32)).sum(-1)

def pairs_distance_float(X,Y):
    return abs((X[:, None, :] - Y).astype(npj.float32)).sum(-1)

In [17]:
vector_distance_jit = jit(vector_distance_float)
pairwise_distance_jit = jit(pairwise_distance_float)
pairs_distance_jit = jit(pairs_distance_float)

In [ ]:
times_list = []
N = 2
for i in range(N):
    print("{}/{}     ".format(i+1,N), end='\r')
    ts = [0, 0, 0]
    np.random.shuffle(M)
    M = deepcopy(transformer(M.astype(np.float64)).astype(np.int8))
    vec = deepcopy(M[:100])
    t = time()
    _ = pairs_distance(vec,M)
    ts[0] = time()-t
    t = time()
    _ = pairs_distance_pythran(vec,M)
    ts[1] = time()-t
    A = M.astype(npj.float32)
    t = time()
    _ = pairs_distance_jit(vec,M)
    ts[2] = time()-t
    times_list.append(ts)

In [ ]:
print(pd.DataFrame(times_list, columns=['Pure','Pythran','JAX/JIT']).describe()) 

In [ ]:
0.056382*460000

In [ ]:

Uid = NewType('uid', int)
Raw = NewType('raw', str)

class ExSVD():
    """
        Classe para SVD.
        
    
    """
    def __init__(self,stateless: bool = False):
        self.matrix_dict = {}
        self.matrix_dict_2 = {}
        self.stateless = stateless

    def fit(self,trainset = None):
        """
            Reimplementei a SVD.fit para colocar um logger nível INFO.
        """
        #logger.info("Treinando modelo SVD...")
        #logger.info("Pronto!")
    
    def _get_neighbors(self,uid: Uid, k: int = 1, black_list: List[Uid] = []) -> List[Uid]:
        """
            Calcula todas as distâncias entre 'uid' de entrada e todos os outros 'uid'.
            A distância calciulada é armazenda e não calculada novamente. 
        """
        black_list.append(uid)
        k = k if k >= 0 else 0
        #logger.info("Calculando todos os vizinhos...")
        #for uid2 in tqdm(self.trainset.all_users()):
        for uid2 in self.trainset.all_users():
            ordered = tuple(sorted((uid,uid2)))
            if (uid2 not in black_list) and (ordered not in self.matrix_dict.keys()):
                self.matrix_dict[ordered] = cosine(self.pu[uid],self.pu[uid2])
        out = [x[0] for x in sorted(
            [
                (uid2, self.matrix_dict[tuple(sorted((uid,uid2)))]) 
                for uid2 in self.trainset.all_users()
                if (uid2 not in black_list)
            ], key=lambda x: x[1])][:k-1]
        if self.stateless:
            del self.matrix_dict
            self.matrix_dict = {}
        return out

    def _get_neighbors_2(self,uid: Uid, k: int = 1, black_list: List[Uid] = []) -> List[Uid]:
        """
            Calcula todas as distâncias entre 'uid' de entrada e todos os outros 'uid'.
            A distância calciulada é armazenda e não calculada novamente. 
        """
        black_list.append(uid)
        k = k if k >= 0 else 0
        #logger.info("Calculando todos os vizinhos...")
        #for uid2 in tqdm(self.trainset.all_users()):
        if uid not in self.matrix_dict_2.keys():
            Un = transformer(self.pu).astype(np.int8)
            self.matrix_dict_2[uid] = vector_distance_pythran(Un,Un[uid])
        out = [x[0] for x in sorted(
            [
                (uid2, self.matrix_dict_2[uid][uid2])
                for uid2 in self.trainset.all_users()
                if (uid2 not in black_list)
            ], key=lambda x: x[1])][:k-1]
        if self.stateless:
            del self.matrix_dict
            self.matrix_dict = {}
        return out
    
    def _uid2raw(self, uid: Uid)-> str:
        '''
            uid -> raw.
            Valor interno para externo, o nome original do usuário.
        '''
        return self.trainset.to_raw_uid(uid)
    
    def _raw2uid(self, raw: Raw)-> int:
        '''
            raw -> uid.
            Valor externo para interno, o id interno do usuários..
        '''
        return self.trainset.to_inner_uid(raw)
    
    def recomender(self, in_list: List[Raw], k: int = 1, L: int = 3, Fk: int = 1, limit: int = 100)-> List[Raw]:
        '''
            Faz as recomendacoes.
            ##### Função incompleta #####
        '''
        # Pega quantas recomendações por usuário em `in_list`,
        # mas sem deixar faltar
        N_in = len(in_list)
        k = k if k > 0 else 1
        R_per_in = L*(k//N_in + min(k%N_in,1))

        # Pega os `uid`
        uid_in_list = []
        for raw in in_list:
            uid_in_list.append(self._raw2uid(raw))

        # Pega os vizinhos mais próximos de cada uid de entrada.
        done = False
        flag = True
        Rounds = 0
        while limit and (not done):
            Rounds += 1
            # Ele sempre pega todos novamente.
            recomendations_list = []
            for i,uid in enumerate(uid_in_list):
                #logger.info("Calculando todos os vizinhos...{:,}/{:,} (Round: {:,}).".format(i+1,N_in,Rounds))
                #recomendations_list.append(self._get_neighbors(uid,R_per_in,in_list))
                recomendations_list.append(self._get_neighbors_2(uid,R_per_in,in_list))
            # Quando limit = 0, encerra.
            limit -= 1
            # Quando tem gente o suficiente, encerra.
            if len(set(flat(recomendations_list))) >= Fk*k:
                done = True
            # Depois do primeiro loop, pega um a mais.
            R_per_in += 1

        # Aqui gera um dicionário ordenando por votacao.
        count_rec = Counter(flat(recomendations_list)) # A votação!!
        count_rec = list(count_rec.items())
        ct_pos = defaultdict(list)
        #ct_pos_inv = defaultdict(list)
        while count_rec:
            tmp = count_rec.pop(0)
            ct_pos[tmp[1]].append(tmp[0])
            #ct_pos_inv[tmp[0]].append(tmp[1])

        # Aqui considera a posiçao de vizinhos mais proximos.
        #nn_pos = defaultdict(list)
        nn_pos_inv = defaultdict(list)
        tmp = deepcopy(recomendations_list)
        while tmp:
            tmp2 = tmp.pop(0)
            n = 0
            while tmp2:
                n += 1
                tmp3 = tmp2.pop(0)
                #nn_pos[n].append(tmp3)
                nn_pos_inv[tmp3].append(n)

        # Vai separando por votação e ordem de proximidade como desempate.      
        votos_list = list(ct_pos.keys())
        out_uid = []
        while votos_list and k:
            votos = max(votos_list)
            votos_list.remove(votos)
            tmp = sorted([(tmp, min(nn_pos_inv[tmp])) for tmp in ct_pos[votos]], key=lambda x: x[1])
            while tmp and k:
                out_uid.append(tmp.pop(0)[0])
                k -= 1

        # converte para Raw e "joga fora".
        return [self._uid2raw(uid) for uid in out_uid]


In [ ]:
ex_algo = ExSVD(stateless=True, n_factors=200, n_epochs=20, verbose=True)
ex_algo.fit(data.build_full_trainset())

Passo de validação simples, para cada uma empresa no portfólio pegar N recomendações e ver se uma delas está no portfólio. Se está, soma 1, se não, soma 0.

In [ ]:
N = 10
tmp = []
n = 0
Nu = 10
times = []
for row in df_ep.sample(n=Nu).iterrows():
    t = time()
    n += 1
    print("Empresa {:,}/{:,}.".format(n,Nu), end='\r')
    recs = ex_algo.recomender([row[1].id],k=N)
    tmp.append(any([x in df_ep.loc[df_ep.P == row[1].P].id.to_list() for x in recs])*1)
    times.append(time()-t)

In [ ]:
sum(tmp)

In [ ]:
pd.DataFrame(times, columns=['time']).describe()

**Item 2 pronto**, usei o Pythran e ficou bem mais rápido, mas o passo de treino não está.
Creio que é chegado o momento de jogar fora o Surprise antes de implementar a busca por hiperparâmetros. 